In [92]:
import stim
import pymatching


DISTANCE = 3
ROUNDS = 3
SHOTS = int(1e5)

after_clifford_depolarization = 2e-2
after_reset_flip_probability = 0.1e-2
before_measure_flip_probability = 0.5e-2 
before_round_data_depolarization = 0.8e-2

circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=after_clifford_depolarization, #two-qubit-fidelity,
                                after_reset_flip_probability=after_reset_flip_probability, #reset error,
                                before_measure_flip_probability=before_measure_flip_probability, #measurement error,
                                before_round_data_depolarization=before_round_data_depolarization) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

# Meas Sampling

In [93]:
meas_outcomes = circuit.compile_sampler().sample(SHOTS)
print(meas_outcomes)
print()
print(meas_outcomes.shape)
print()
print(len(meas_outcomes[0]))
print(DISTANCE + ROUNDS*(DISTANCE-1))

[[False False False ... False False False]
 [False False False ... False False False]
 [ True False  True ...  True False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]

(100000, 9)

9
9


In [94]:
counts = {}
for row in meas_outcomes:
    count_str = ''
    for nb, bit in enumerate(row):
        count_str += '0' if bit == False else '1'
        if (nb+1) % (DISTANCE-1) == 0 and nb < ROUNDS*(DISTANCE-1):
            count_str += ' ' 
    
    count_str = count_str[::-1]
    if count_str in counts:
        counts[count_str] += 1
    else:
        counts[count_str] = 1
print(counts)

{'000 00 00 00': 74520, '001 11 01 01': 18, '100 10 00 00': 1187, '000 00 01 00': 1363, '000 00 00 10': 1254, '010 00 00 00': 733, '000 01 00 00': 1247, '100 00 00 00': 737, '010 00 00 01': 14, '000 10 00 00': 1323, '001 01 01 00': 1302, '001 01 01 01': 842, '000 00 11 01': 16, '000 00 10 00': 1319, '010 01 00 00': 852, '110 01 01 00': 29, '010 11 00 01': 24, '010 11 00 00': 1227, '110 11 10 10': 14, '101 11 10 00': 27, '001 00 00 10': 14, '010 11 01 00': 876, '100 10 10 00': 1216, '001 01 00 00': 1263, '001 11 01 00': 25, '000 00 11 00': 34, '010 11 11 11': 893, '001 00 00 00': 766, '000 00 00 01': 1265, '000 01 01 00': 39, '010 11 11 00': 1242, '011 10 10 10': 16, '101 10 10 10': 12, '010 11 11 01': 862, '101 11 11 00': 36, '110 11 11 01': 8, '101 11 01 00': 26, '110 11 00 00': 27, '010 11 11 10': 35, '100 10 10 10': 936, '000 00 10 10': 36, '000 00 01 01': 41, '110 11 10 11': 1, '110 01 10 10': 13, '101 11 01 01': 13, '011 10 10 00': 19, '010 00 01 00': 13, '010 10 00 00': 40, '100 

In [116]:
circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=after_clifford_depolarization, #two-qubit-fidelity,
                                after_reset_flip_probability=after_reset_flip_probability, #reset error,
                                before_measure_flip_probability=0.5e-2, #measurement error,
                                before_round_data_depolarization=before_round_data_depolarization) #idle error)


model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [117]:
# decode the meas outcomes
from cpp_soft_info import counts_to_det_syndr

num_err = 0
for count_key, val in counts.items():
    syndr = counts_to_det_syndr(count_key, _resets = True, verbose = False)
    predicted_flip = matching.decode(syndr)
    # print(count_key[0])
    # print(predicted_flip[0])
    if int(predicted_flip[0]) != int(count_key[0]):
        num_err += val

print(f"Num errors: {num_err} out of {sum(counts.values())} shots")

Num errors: 806 out of 100000 shots


# Detector sampling

In [96]:
syndromes = circuit.compile_detector_sampler().sample(SHOTS, append_observables=True)
print(syndromes.shape)
print(syndromes)



(100000, 9)
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False  True]]


### Generate second stim for decoding

In [97]:
circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=after_clifford_depolarization, #two-qubit-fidelity,
                                after_reset_flip_probability=after_reset_flip_probability, #reset error,
                                before_measure_flip_probability=before_measure_flip_probability, #measurement error,
                                before_round_data_depolarization=before_round_data_depolarization) #idle error)


model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [98]:
num_err = 0

for i in range(len(syndromes))[:]:
    predicted_flip = matching.decode(syndromes[i][:-1])
    if bool(predicted_flip[0]) != (syndromes[i][-1]):
        num_err += 1

print("Error rate:", num_err, "out of:", len(syndromes), "total")  



Error rate: 824 out of: 100000 total
